In [44]:
import tensorflow_decision_forests as tfdf
import src.data.utils
import os
import sys
import numpy as np
import datatable as dt
from datatable import f
import pandas as pd
import tensorflow as tf
import math
import random
import scanpy as sc
from scipy import sparse

# Generate the dataset

In [2]:
anno = dt.fread("../data/raw/Doubletannotation.csv")
# expr = dt.fread("../data/raw/pbmc_expr_t.csv")
expr = dt.fread("../data/processed/pbmc_expr_t_filtered_12000_30000.csv")
# expr_reduced = pd.read_csv("../data/processed/pbmc_expr_reduced.csv", index_col=0)
# expr_reduced.drop("C0", axis=1, inplace=True)

In [4]:
full = sc.read_10x_h5("../data/raw/pbmc_100hvg.h5ad")

ValueError: '../data/raw/pbmc_100hvg.h5ad' contains more than one genome. For legacy 10x h5 files you must specify the genome if more than one is present. Available genomes are: ['X', 'layers', 'obs', 'obsm', 'obsp', 'uns', 'var', 'varm', 'varp']

In [46]:
X = pd.DataFrame.sparse.from_spmatrix(full.X)

In [52]:
X.index = full.obs_names

In [55]:
X.columns = full.var_names

In [56]:
X

,TGFBR3,FCRL1,FCGR3A,RALGPS2,AL136456.1,RHEX,RGS7,GNLY,AFF3,IL1B,...,NKG7,TSHZ2,AL109930.1,IGLC1,IGLC2,IGLC3,SYN3,LARGE1,AC108879.1,EDA
AAACAGCCAAAGCCTC-1,0.0,0.0,0.0,0.000000,0.0,0.0,1.184205,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,1.184205,0.0,0.0,0.0,0.0,0.0
AAACAGCCAGAATGAC-1,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,1.048871,0.0,...,0.656075,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
AAACAGCCAGCTACGT-1,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
AAACAGCCAGGCCTTG-1,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,1.984647,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
AAACAGCCATAAGGAC-1,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTACGCGCA-1,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,2.448487,0.0,...,0.000000,1.838231,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
TTTGTTGGTATTTGCC-1,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,1.281151,0.000000,0.0,...,1.281151,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
TTTGTTGGTGATTACG-1,0.0,0.0,0.0,1.717464,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
TTTGTTGGTTTCAGGA-1,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [37]:
scanpy.get.var_df(full)

""
TGFBR3
FCRL1
FCGR3A
RALGPS2
AL136456.1
...
IGLC3
SYN3
LARGE1
AC108879.1


In [4]:
expr

,C11,C10,OR4F5,AL627309.1,AL627309.3,AL627309.5,AL627309.4,AP006222.2,AL669831.2,LINC01409,…,AC233755.1,AC136616.2,AC007325.1,AC007325.4,AC007325.2
,▪▪▪▪,▪▪▪▪,▪,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪,▪,▪▪▪▪,,▪,▪,▪,▪,▪
0,0,AAACAGCCAAAGCCTC.1,0,0,0,0,0,0,0,0,…,0,0,0,0,0
1,1,AAACAGCCAGAATGAC.1,0,0,0,0,0,0,0,0,…,0,0,0,0,0
2,2,AAACAGCCAGCTACGT.1,0,0,0,0,0,0,0,0,…,0,0,0,0,0
3,3,AAACAGCCAGGCCTTG.1,0,0,0,0,0,0,0,0,…,0,0,0,0,0
4,4,AAACAGCCATAAGGAC.1,0,0,0,0,0,0,0,0,…,0,0,0,0,0
5,5,AAACAGCCATAATGTC.1,0,0,0,0,0,0,0,0,…,0,0,0,1,0
6,6,AAACATGCAGCAATAA.1,0,0,0,0,0,0,0,0,…,0,0,0,0,0
7,7,AAACATGCAGCCAGAA.1,0,0,0,0,0,0,0,0,…,0,0,0,0,0
8,8,AAACATGCAGGCGATA.1,0,0,0,0,0,0,0,0,…,0,0,0,0,0


In [ ]:
anno = expr.to_pandas()
expr = expr.to_pandas()

In [6]:
expr.shape

(12012, 30351)

Filter genes with only one value in every row

In [7]:
expr = expr.drop("C0", axis=1, error="ignore")

In [9]:
expr = expr.drop("C11", axis=1, errors="ignore")

In [10]:
expr.to_csv("../data/processed/pbmc_expr_t_filtered_12000_30000.csv")

In [4]:
random.seed(1234)
expr_reduced = expr.drop(expr.columns[np.random.choice(range(1, expr.shape[1]), size=24000, replace=False)], axis=1)
expr_reduced = expr_reduced.drop(np.random.choice(range(1, expr.shape[0]), size=8000, replace=False), axis=0)

In [15]:
expr_reduced.to_csv("../data/processed/pbmc_expr_reduced.csv")

In [ ]:
expr.iloc[:, 1:] = expr.iloc[:, 1:].apply(pd.to_numeric, errors="ignore")

In [18]:

anno["cell_type"] = anno.apply(lambda row: class_checker(row), axis=1)
anno_filtered = anno[["V1", "cell_type"]]
anno = anno_filtered.rename(columns={"V1": "C0"})
anno_filtered.rename(columns={"V1": "C10"}, inplace=True)
anno_filtered

In [12]:
expr_reduced.rename(columns={"C10": "C0"}, inplace=True)
expr_reduced

,C0,MIR1302-2HG,FAM138A,OR4F29,AC114498.1,LINC01409,AL645608.6,AL645608.4,LINC02593,NOC2L,...,AC011043.2,BX072566.1,AL354822.1,AC240274.1,AC213203.2,AC136352.3,AC136352.2,AC171558.3,AC007325.2,C0
0,AAACAGCCAAAGCCTC.1,False,False,False,False,0,False,False,False,0,...,False,False,False,1,False,False,False,False,False,AAACAGCCAAAGCCTC.1
6,AAACATGCAGCAATAA.1,False,False,False,False,0,False,False,False,0,...,False,False,False,0,False,False,False,False,False,AAACATGCAGCAATAA.1
8,AAACATGCAGGCGATA.1,False,False,False,False,0,False,False,False,0,...,False,False,False,0,False,False,False,False,False,AAACATGCAGGCGATA.1
13,AAACCAACAGGACCTT.1,False,False,False,False,1,False,False,False,0,...,False,False,False,0,False,False,False,False,False,AAACCAACAGGACCTT.1
19,AAACCGAAGCGTGCGT.1,False,False,False,False,0,False,False,False,0,...,False,False,False,0,False,False,False,False,False,AAACCGAAGCGTGCGT.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12001,TTTGTGGCATAAGGAC.1,False,False,False,False,0,False,False,False,0,...,False,False,False,0,False,False,False,False,False,TTTGTGGCATAAGGAC.1
12003,TTTGTGTTCAGTTCCC.1,False,False,False,False,0,False,False,False,0,...,False,False,False,0,False,False,False,False,False,TTTGTGTTCAGTTCCC.1
12008,TTTGTTGGTATTTGCC.1,False,False,False,False,0,False,False,False,0,...,False,False,False,0,False,False,False,False,False,TTTGTTGGTATTTGCC.1
12010,TTTGTTGGTTTCAGGA.1,False,False,False,False,0,False,False,False,0,...,False,False,False,0,False,False,False,False,False,TTTGTTGGTTTCAGGA.1


In [59]:
dic = {"A": 2}
dic.update({"B": 3})

In [63]:
"A" in dic.keys()

True

In [67]:
data = None
assert is.

AssertionError: 

In [20]:
merged = pd.merge(expr_reduced, anno_filtered)

In [21]:
merged

,C10,FAM138A,AL627309.1,AL627309.4,AP006222.2,OR4F29,OR4F16,LINC01409,LINC00115,AL645608.2,...,AC011841.1,BX072566.1,AL354822.1,AC213203.1,AC233755.1,AC136612.1,AC136616.3,AC141272.1,AC023491.2,cell_type
0,AAACAGCCAAAGCCTC.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,singlet
1,AAACAGCCAGAATGAC.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,singlet
2,AAACAGCCAGCTACGT.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,singlet
3,AAACATGCAGCAATAA.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,singlet
4,AAACATGCAGTTTCTC.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,singlet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4007,TTTGTGGCAATGCGCT.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,singlet
4008,TTTGTGGCACGAATTT.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,singlet
4009,TTTGTGGCAGGCAAGC.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,singlet
4010,TTTGTGTTCAGTTCCC.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,singlet
